In [ ]:
import sys
sys.path.append('../python_packages_static/')
import pyemu
import os
import shutil
import numpy as np
import pandas as pd

# Based on the histograms and the PDC file for the prior Monte Carlo, we determine we need to expand bounds on the bedrock parameters and re-evaluate the Monte Carlo

In [ ]:
run_ensemble=False

In [ ]:
if run_ensemble==True:
    template_dir = '../run_data' # read in the prior_mc.pst file from here
    noptmax0_dir = '../noptmax0_testing' # write the new file here to be able to run noptmax=0 and reqweight again
else:
    template_dir = '../output/noptmax0/'
    noptmax0_dir = '../output/noptmax0/'

In [ ]:
pst=pyemu.Pst(os.path.join(template_dir,'prior_mc.pst'))
update_forward_run=False
run_local=True

## update some parameter bounds

In [ ]:
pars = pst.parameter_data

### K-zones set to not get too crazy high

In [ ]:
#  read in k value lookup table to df

#  original table

k_df_original = pd.read_excel(
    '../processed_data/Rondout_Neversink_GeologyLookupTable.xlsx',
    sheet_name='Sheet2'
)
k_df_original.index = k_df_original.Lookup_Code

k_df = pd.read_excel(
    '../processed_data/Rondout_Neversink_GeologyLookupTable_jhw.xlsx',
    sheet_name='Sheet2'
)

k_df.index = k_df.Lookup_Code

print('Using mean K value')
k_df['Kh_ft_d_mean'] = (k_df['Kh_ft_d_lower'] + k_df['Kh_ft_d_upper']) / 2
k_df['Kh_m_d'] = k_df['Kh_ft_d_mean'] * 0.3048
    
k_df['Kh_m_d_lower'] = k_df['Kh_ft_d_lower'] * .3048
k_df['Kh_m_d_upper'] = k_df['Kh_ft_d_upper'] * .3048

k_df['K_upper_mult'] = k_df['Kh_m_d_upper'] / k_df['Kh_m_d']
k_df['K_lower_mult'] = k_df['Kh_m_d_lower'] / k_df['Kh_m_d']

k_df

### find the ratio of new expanded K values to the old

In [ ]:
k_df['new_ratio']=k_df_original.Kh_m_d/k_df.Kh_m_d
k_df

In [ ]:
k_df['new_ratio'] = [min(i,50) for i in k_df.new_ratio] # cap the expansion of bounds at 50
k_df

In [ ]:
k_mult_zones = [int(i.split(':')[-1]) for i in pars.loc[pars.parnme.str.startswith('multiplier_k')].index]
np.unique(k_mult_zones)

In [ ]:
k_mult_zones

### apply the new bounds to the k-zone multipliers

In [ ]:
new_mults = [k_df.loc[i].new_ratio for i in k_mult_zones]
upper_mults = np.array(new_mults)*100
lower_mults = np.array(new_mults)/100

In [ ]:
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parlbnd'] = lower_mults
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parubnd'] = upper_mults
pars.loc[pars.parnme.str.startswith('multiplier_k'), 'parval1'] = new_mults


### pilot points set to mean upper and lower bounds diffs

In [ ]:
mean_lower = k_df.K_lower_mult.mean()
mean_upper = k_df.K_upper_mult.mean()
mean_lower,mean_upper

In [ ]:
pars.loc[pars.pargp.str.startswith('k'), 'parlbnd'] = mean_lower + 0.01
pars.loc[pars.pargp.str.startswith('k'), 'parubnd'] = mean_upper - 0.01
pars.loc[pars.pargp.str.startswith('sfrk'), 'parlbnd'] = 0.1
pars.loc[pars.pargp.str.startswith('sfrk'), 'parubnd'] = 10.0
pars.loc[pars.pargp=='chd', 'partrans'] = 'fixed'

### write out an excel report of the expanded parameter ensemble

In [ ]:
parsum = pst.write_par_summary_table('/expanded_parsum.xlsx', report_in_linear_space=True)
parsum

### set noptmax = 0 and write out PST file

In [ ]:
pst.control_data.noptmax = 0

In [ ]:
pst.write(os.path.join(noptmax0_dir,'prior_mc_wide.pst'))

### again, for windows to be moved to linux for running, remove backslashes from pest ctrl file

In [ ]:
if sys.platform == 'win32':
    f = open(os.path.join(template_dir, 'prior_mc_wide.pst'), "r")
    lines = f.readlines()
    f.close()

    output_lines = []
    for line in lines:
        output_lines.append(line.replace('\\', "/"))

    f = open(os.path.join(template_dir, 'prior_mc_wide.pst'), "w")
    f.write(''.join(output_lines))
    f.close()